In [4]:
import pandas as pd
from datetime import datetime
import numpy as np


In [2]:
def convert_date(date_str):
    try:
        date_obj = datetime.strptime(str(date_str), '%a %b %d %H:%M:%S %z %Y')
        return date_obj.strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        return np.nan
    
def convert_date_df(df):
    df['date'] = df['created_at'].apply(convert_date)
    df = df.dropna(subset=['date'])
    df['date'] = pd.to_datetime(df['date'])

    # Create 'year' and 'month' columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    return df


def filter_emotion(df):
    df = df[['screen_name','joy_pys','sadness_pys','anger_pys','surprise_pys','disgust_pys','fear_pys','date']]
    df['date'] = pd.to_datetime(df['date'])
    return df

def filter_emotion_all(df):
    df = df[['screen_name','joy','sadness','anger','surprise','disgust','fear','date']]
    df['date'] = pd.to_datetime(df['date'])
    return df
def avg_emotion_per_user(df):
    df.loc[:, 'date'] = pd.to_datetime(df['date'])

    df.set_index('date', inplace=True)
    # Resample DataFrame to weekly frequency and calculate mean for each user
    weekly_avg_per_user = df.groupby('screen_name').resample('M').mean()#w
    # print(weekly_avg_per_user)

    # Calculate the overall weekly average by taking the mean of the weekly averages per user
    # overall_weekly_avg = weekly_avg_per_user.mean(level='date')
    overall_weekly_avg = weekly_avg_per_user.groupby(level='date').mean()

    return overall_weekly_avg


def replace_column_emotion(df):
    df.columns = df.columns.str.replace('_pys', '', regex=False)
    return df


In [3]:
HPS = pd.read_csv('///home/noaai/HCP_State_of_Mind_covid_19/pysentimento_tweets_transformer_and_optim/new/job1_tweets_emotion_pysentimento.csv')
HPS = convert_date_df(HPS)
HPS = filter_emotion(HPS)
HPS = replace_column_emotion(HPS)

unique_screen_names = pd.read_csv('../active_h_90.csv')['screen_name'].tolist()
final_h = HPS[HPS['screen_name'].isin(unique_screen_names)]

/tmp/ipykernel_150465/387501580.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  HPS = pd.read_csv('///home/noaai/HCP_State_of_Mind_covid_19/pysentimento_tweets_transformer_and_optim/new/job1_tweets_emotion_pysentimento.csv')
/tmp/ipykernel_150465/2725092837.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/tmp/ipykernel_150465/2725092837.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df

In [5]:
file_paths = ['job3_tweets_emotion_pysentimento.csv',
              'job2_tweets_emotion_pysentimento.csv',
              'job13_tweets_emotion_pysentimento.csv',
              'job4_tweets_emotion_pysentimento.csv',
              'job1_tweets_emotion_pysentimento.csv',
              'job12_tweets_emotion_pysentimento.csv',
              'job10_tweets_emotion_pysentimento.csv',
              'job11_tweets_emotion_pysentimento.csv',
              'job7_tweets_emotion_pysentimento.csv',
              'job5_tweets_emotion_pysentimento.csv',
              'job8_tweets_emotion_pysentimento.csv',
              'job6_tweets_emotion_pysentimento.csv',
              'job9_tweets_emotion_pysentimento.csv',
             'job18_tweets_emotion_pysentimento.csv']
OUTPUT_FILE_PATH = '/home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/'
# Read each file into a DataFrame and store it in a list
dfs = [pd.read_csv(OUTPUT_FILE_PATH+file_path) for file_path in file_paths]

# Concatenate all the DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

In [6]:
general = convert_date_df(combined_df)
general = filter_emotion(general)
general = replace_column_emotion(general)

/tmp/ipykernel_150465/2725092837.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [7]:
final_df = general

In [8]:
final_h.loc[:, 'date'] = pd.to_datetime(final_h['date'])
final_h.set_index('date', inplace=True)

final_df.loc[:, 'date'] = pd.to_datetime(final_df['date'])
final_df.set_index('date', inplace=True)


In [10]:
hcp_avg_user = final_h.reset_index()
general_avg_user = final_df.reset_index()


In [11]:
import pandas as pd
import numpy as np
from scipy.stats import t

# # Define the periods according to the dates
before_phase_start = '2019-01-01'  # Assuming a start date for the "before" period
before_phase_end = '2020-02-29'
phase_1_start = '2020-03-01'
phase_1_end = '2020-10-31'
phase_2_start = '2020-11-01'
phase_2_end = '2021-04-30'
phase_3_start = '2021-05-01'
phase_3_end = '2021-12-31'
phase_4_start = '2022-01-01'
phase_4_end = '2022-05-31'


# Function to categorize each date into a phase
def categorize_phase(date):
    if before_phase_start <= date <= before_phase_end:
        return 'Before COVID Phases'
    elif phase_1_start <= date <= phase_1_end:
        return 'Phase 1'
    elif phase_2_start <= date <= phase_2_end:
        return 'Phase 2'
    elif phase_3_start <= date <= phase_3_end:
        return 'Phase 3'
    elif phase_4_start <= date <= phase_4_end:
        return 'Phase 4'
    else:
        return 'Outside COVID Phases'

# Apply the categorize function to add a 'phase' column
hcp_avg_user['phase'] = hcp_avg_user['date'].astype(str).apply(categorize_phase)
general_avg_user['phase'] = general_avg_user['date'].astype(str).apply(categorize_phase)


In [12]:
hcp_avg_by_phase = hcp_avg_user.groupby(['screen_name', 'phase']).mean()
general_avg_by_phase = general_avg_user.groupby(['screen_name', 'phase']).mean()

hcp_avg_by_phase = hcp_avg_by_phase.reset_index()
general_avg_by_phase = general_avg_by_phase.reset_index()


In [13]:
hcp_phase_data = hcp_avg_by_phase[hcp_avg_by_phase['phase'] != 'Outside COVID Phases']# !1!!!!!!!!!!!!!!!!!!!!!!!
general_phase_data = general_avg_by_phase[general_avg_by_phase['phase'] != 'Outside COVID Phases']


In [33]:
import pandas as pd
import numpy as np
from scipy.stats import t



# Function to calculate the 95% confidence interval lower and upper bounds
def calculate_95ci(series):
    n = series.count()
    if n > 1:
        mean = series.mean()
        se = series.std() / np.sqrt(n)
        ci95 = t.ppf(0.975, df=n-1) * se
        lower_bound = mean - ci95
        upper_bound = mean + ci95
        return lower_bound, upper_bound
    else:
        return np.nan, np.nan  # If there's not enough data to calculate a CI

# Compute statistics for each phase for HCP populations
hcp_stats = hcp_phase_data.groupby('phase').agg({
    'screen_name': 'nunique',  # Unique count of users
    'joy': ['mean', 'std', calculate_95ci],
    'sadness': ['mean', 'std', calculate_95ci],
    'anger': ['mean', 'std', calculate_95ci],
    'surprise': ['mean', 'std', calculate_95ci],
    'disgust': ['mean', 'std', calculate_95ci],
    'fear': ['mean', 'std', calculate_95ci]
}).reset_index()

# Compute statistics for each phase for General populations
general_stats = general_phase_data.groupby('phase').agg({
    'screen_name': 'nunique',  # Unique count of users
    'joy': ['mean', 'std', calculate_95ci],
    'sadness': ['mean', 'std', calculate_95ci],
    'anger': ['mean', 'std', calculate_95ci],
    'surprise': ['mean', 'std', calculate_95ci],
    'disgust': ['mean', 'std', calculate_95ci],
    'fear': ['mean', 'std', calculate_95ci]
}).reset_index()

# Rename columns for clarity
hcp_stats.columns = ['phase', 'unique_users', 
                     'joy_mean', 'joy_std', 'joy_95ci',
                     'sadness_mean', 'sadness_std', 'sadness_95ci',
                     'anger_mean', 'anger_std', 'anger_95ci',
                     'surprise_mean', 'surprise_std', 'surprise_95ci',
                     'disgust_mean', 'disgust_std', 'disgust_95ci',
                     'fear_mean', 'fear_std', 'fear_95ci']

general_stats.columns = ['phase', 'unique_users', 
                         'joy_mean', 'joy_std', 'joy_95ci',
                         'sadness_mean', 'sadness_std', 'sadness_95ci',
                         'anger_mean', 'anger_std', 'anger_95ci',
                         'surprise_mean', 'surprise_std', 'surprise_95ci',
                         'disgust_mean', 'disgust_std', 'disgust_95ci',
                         'fear_mean', 'fear_std', 'fear_95ci']



# Save Data

In [35]:
hcp_stats.to_pickle('Cohen_d/hcp_stats_cohend.pkl')

general_stats.to_pickle('Cohen_d/nonhcp_cohend.pkl')


# Load Data

In [36]:
# # hcp_stats = pd.read_csv('Cohen_d/hcp_stats_cohend.csv')
# # general_stats = pd.read_csv('Cohen_d/nonhcp_cohend.csv.csv')

# # Load HCP stats from Excel
# hcp_stats = pd.read_excel('Cohen_d/hcp_stats_with_cohend.xlsx')

# # Load General Population stats from CSV
# general_stats = pd.read_csv('Cohen_d/general_stats_with_cohend.csv')
hcp_stats = pd.read_pickle('Cohen_d/hcp_stats_cohend.pkl')
general_stats = pd.read_pickle('Cohen_d/nonhcp_cohend.pkl')

In [37]:
# Formatting output to match the style of the image
def format_output(stats_df, group_name):
    for index, row in stats_df.iterrows():
        print(f"The top 5 themes for {group_name} in {row['phase']} were:")
        print(f"- Joy: 95% CI [{row['joy_95ci'][0]:.3f}, {row['joy_95ci'][1]:.3f}]")
        print(f"- Sadness: 95% CI [{row['sadness_95ci'][0]:.3f}, {row['sadness_95ci'][1]:.3f}]")
        print(f"- Anger: 95% CI [{row['anger_95ci'][0]:.3f}, {row['anger_95ci'][1]:.2f}]")
        print(f"- Surprise: 95% CI [{row['surprise_95ci'][0]:.3f}, {row['surprise_95ci'][1]:.3f}]")
        print(f"- Disgust: 95% CI [{row['disgust_95ci'][0]:.3f}, {row['disgust_95ci'][1]:.3f}]")
        print(f"- Fear: 95% CI [{row['fear_95ci'][0]:.3f}, {row['fear_95ci'][1]:.3f}]\n")

# Output the formatted statistics
format_output(hcp_stats, 'HCP Population')
format_output(general_stats, 'General Population')


The top 5 themes for HCP Population in Before COVID Phases were:
- Joy: 95% CI [0.238, 0.253]
- Sadness: 95% CI [0.026, 0.028]
- Anger: 95% CI [0.014, 0.02]
- Surprise: 95% CI [0.016, 0.017]
- Disgust: 95% CI [0.075, 0.084]
- Fear: 95% CI [0.016, 0.018]

The top 5 themes for HCP Population in Phase 1 were:
- Joy: 95% CI [0.200, 0.214]
- Sadness: 95% CI [0.032, 0.035]
- Anger: 95% CI [0.016, 0.02]
- Surprise: 95% CI [0.014, 0.015]
- Disgust: 95% CI [0.080, 0.089]
- Fear: 95% CI [0.034, 0.038]

The top 5 themes for HCP Population in Phase 2 were:
- Joy: 95% CI [0.222, 0.237]
- Sadness: 95% CI [0.030, 0.034]
- Anger: 95% CI [0.016, 0.02]
- Surprise: 95% CI [0.014, 0.016]
- Disgust: 95% CI [0.077, 0.086]
- Fear: 95% CI [0.023, 0.026]

The top 5 themes for HCP Population in Phase 3 were:
- Joy: 95% CI [0.236, 0.252]
- Sadness: 95% CI [0.029, 0.032]
- Anger: 95% CI [0.013, 0.02]
- Surprise: 95% CI [0.014, 0.015]
- Disgust: 95% CI [0.068, 0.076]
- Fear: 95% CI [0.022, 0.025]

The top 5 themes

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import t

# Function to calculate Cohen's d
def calculate_cohen_d(mean1, mean2, std1, std2, n1, n2):
    pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
    d = (mean1 - mean2) / pooled_std
    return d

# Extract 'Before COVID Phases' data
hcp_before_data = hcp_stats[hcp_stats['phase'] == 'Before COVID Phases']
general_before_data = general_stats[general_stats['phase'] == 'Before COVID Phases']

# List of emotions to compute Cohen's d
emotions = ['sadness', 'anger', 'surprise', 'disgust', 'fear', 'joy']

# Compute Cohen's d for HCP populations
for emotion in emotions:
    hcp_stats[f'{emotion}_cohen_d'] = hcp_stats.apply(lambda row: calculate_cohen_d(
        row[f'{emotion}_mean'], 
        hcp_before_data[f'{emotion}_mean'].mean(),
        row[f'{emotion}_std'],
        hcp_before_data[f'{emotion}_std'].mean(),
        row['unique_users'],
        hcp_before_data['unique_users'].sum()
    ) if row['phase'] != 'Before COVID Phases' else np.nan, axis=1)

# Compute Cohen's d for General populations
for emotion in emotions:
    general_stats[f'{emotion}_cohen_d'] = general_stats.apply(lambda row: calculate_cohen_d(
        row[f'{emotion}_mean'], 
        general_before_data[f'{emotion}_mean'].mean(),
        row[f'{emotion}_std'],
        general_before_data[f'{emotion}_std'].mean(),
        row['unique_users'],
        general_before_data['unique_users'].sum()
    ) if row['phase'] != 'Before COVID Phases' else np.nan, axis=1)

# Function to format and print the results
def format_output(stats_df, group_name):
    for index, row in stats_df.iterrows():
        print(f"The top 5 themes for {group_name} in {row['phase']} were:")
        for emotion in emotions:
            cohen_d_value = row[f'{emotion}_cohen_d']
            ci_lower, ci_upper = row[f'{emotion}_95ci']
            print(f"- {emotion.capitalize()}: 95% CI [{ci_lower:.3f}, {ci_upper:.3f}], Cohen's d: {cohen_d_value:.3f}")
        print()

# Output the formatted statistics with Cohen's d
format_output(hcp_stats, 'HCP Population')
format_output(general_stats, 'General Population')


The top 5 themes for HCP Population in Before COVID Phases were:
- Sadness: 95% CI [0.026, 0.028], Cohen's d: nan
- Anger: 95% CI [0.014, 0.017], Cohen's d: nan
- Surprise: 95% CI [0.016, 0.017], Cohen's d: nan
- Disgust: 95% CI [0.075, 0.084], Cohen's d: nan
- Fear: 95% CI [0.016, 0.018], Cohen's d: nan
- Joy: 95% CI [0.238, 0.253], Cohen's d: nan

The top 5 themes for HCP Population in Phase 1 were:
- Sadness: 95% CI [0.032, 0.035], Cohen's d: 0.233
- Anger: 95% CI [0.016, 0.019], Cohen's d: 0.072
- Surprise: 95% CI [0.014, 0.015], Cohen's d: -0.128
- Disgust: 95% CI [0.080, 0.089], Cohen's d: 0.061
- Fear: 95% CI [0.034, 0.038], Cohen's d: 0.731
- Joy: 95% CI [0.200, 0.214], Cohen's d: -0.274

The top 5 themes for HCP Population in Phase 2 were:
- Sadness: 95% CI [0.030, 0.034], Cohen's d: 0.175
- Anger: 95% CI [0.016, 0.020], Cohen's d: 0.079
- Surprise: 95% CI [0.014, 0.016], Cohen's d: -0.112
- Disgust: 95% CI [0.077, 0.086], Cohen's d: 0.024
- Fear: 95% CI [0.023, 0.026], Cohen'

In [23]:
# Function to calculate Cohen's d and its 95% confidence interval
def calculate_cohen_d_and_ci(mean1, mean2, std1, std2, n1, n2, alpha=0.05):
    # Calculate Cohen's d
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    d = (mean1 - mean2) / pooled_std
    
    # Calculate standard error
    se_d = np.sqrt((n1 + n2) / (n1 * n2) + d**2 / (2 * (n1 + n2)))
    
    # Critical value from t-distribution
    t_crit = t.ppf(1 - alpha / 2, df=n1 + n2 - 2)
    
    # Calculate confidence interval
    ci_lower = d - t_crit * se_d
    ci_upper = d + t_crit * se_d
    
    return d, (ci_lower, ci_upper)

# Example usage for HCP populations
for emotion in emotions:
    hcp_stats[f'{emotion}_cohen_d'] = hcp_stats.apply(lambda row: calculate_cohen_d_and_ci(
        row[f'{emotion}_mean'], 
        hcp_before_data[f'{emotion}_mean'].mean(),
        row[f'{emotion}_std'],
        hcp_before_data[f'{emotion}_std'].mean(),
        row['unique_users'],
        hcp_before_data['unique_users'].sum()
    )[0] if row['phase'] != 'Before COVID Phases' else np.nan, axis=1)
    
    hcp_stats[f'{emotion}_cohen_d_ci'] = hcp_stats.apply(lambda row: calculate_cohen_d_and_ci(
        row[f'{emotion}_mean'], 
        hcp_before_data[f'{emotion}_mean'].mean(),
        row[f'{emotion}_std'],
        hcp_before_data[f'{emotion}_std'].mean(),
        row['unique_users'],
        hcp_before_data['unique_users'].sum()
    )[1] if row['phase'] != 'Before COVID Phases' else (np.nan, np.nan), axis=1)


#########################################################################################################

for emotion in emotions:
    general_stats[f'{emotion}_cohen_d'] = general_stats.apply(lambda row: calculate_cohen_d_and_ci(
        row[f'{emotion}_mean'], 
        general_before_data[f'{emotion}_mean'].mean(),
        row[f'{emotion}_std'],
        general_before_data[f'{emotion}_std'].mean(),
        row['unique_users'],
        general_before_data['unique_users'].sum()
    )[0] if row['phase'] != 'Before COVID Phases' else np.nan, axis=1)
    
    general_stats[f'{emotion}_cohen_d_ci'] = general_stats.apply(lambda row: calculate_cohen_d_and_ci(
        row[f'{emotion}_mean'], 
        general_before_data[f'{emotion}_mean'].mean(),
        row[f'{emotion}_std'],
        general_before_data[f'{emotion}_std'].mean(),
        row['unique_users'],
        general_before_data['unique_users'].sum()
    )[1] if row['phase'] != 'Before COVID Phases' else (np.nan, np.nan), axis=1)



# Print results
for index, row in hcp_stats.iterrows():
    print(f"In {row['phase']}:")
    for emotion in emotions:
        cohen_d_value = row[f'{emotion}_cohen_d']
        ci_lower, ci_upper = row[f'{emotion}_cohen_d_ci']
        print(f"- {emotion.capitalize()}: Cohen's d = {cohen_d_value:.3f}, 95% CI = [{ci_lower:.3f}, {ci_upper:.3f}]")
    print()


print("------------------------------")
print("---------------genral---------------")
for index, row in general_stats.iterrows():
    print(f"In {row['phase']}:")
    for emotion in emotions:
        cohen_d_value = row[f'{emotion}_cohen_d']
        ci_lower, ci_upper = row[f'{emotion}_cohen_d_ci']
        print(f"- {emotion.capitalize()}: Cohen's d = {cohen_d_value:.3f}, 95% CI = [{ci_lower:.3f}, {ci_upper:.3f}]")
    print()


In Before COVID Phases:
- Sadness: Cohen's d = nan, 95% CI = [nan, nan]
- Anger: Cohen's d = nan, 95% CI = [nan, nan]
- Surprise: Cohen's d = nan, 95% CI = [nan, nan]
- Disgust: Cohen's d = nan, 95% CI = [nan, nan]
- Fear: Cohen's d = nan, 95% CI = [nan, nan]
- Joy: Cohen's d = nan, 95% CI = [nan, nan]

In Phase 1:
- Sadness: Cohen's d = 0.233, 95% CI = [0.160, 0.306]
- Anger: Cohen's d = 0.072, 95% CI = [-0.001, 0.145]
- Surprise: Cohen's d = -0.128, 95% CI = [-0.201, -0.055]
- Disgust: Cohen's d = 0.061, 95% CI = [-0.012, 0.134]
- Fear: Cohen's d = 0.731, 95% CI = [0.656, 0.806]
- Joy: Cohen's d = -0.274, 95% CI = [-0.347, -0.201]

In Phase 2:
- Sadness: Cohen's d = 0.175, 95% CI = [0.102, 0.248]
- Anger: Cohen's d = 0.079, 95% CI = [0.006, 0.152]
- Surprise: Cohen's d = -0.112, 95% CI = [-0.185, -0.040]
- Disgust: Cohen's d = 0.024, 95% CI = [-0.049, 0.097]
- Fear: Cohen's d = 0.318, 95% CI = [0.245, 0.392]
- Joy: Cohen's d = -0.112, 95% CI = [-0.185, -0.039]

In Phase 3:
- Sadness:

In [ ]:
hcp_stats[[]]

In [25]:
# Compute Cohen's d and CI between HCP and General populations for each phase and emotion
emotions = ['sadness', 'anger', 'surprise', 'disgust', 'fear', 'joy']
cohen_d_comparison = {}

# Iterate over each phase and emotion to compare HCP and General populations
for phase in hcp_stats['phase'].unique():
    for emotion in emotions:
        # Get data for the current phase for both populations
        hcp_data = hcp_stats[hcp_stats['phase'] == phase]
        general_data = general_stats[general_stats['phase'] == phase]
        
        if not hcp_data.empty and not general_data.empty:
            # Calculate mean and std for HCP and General data
            hcp_mean = hcp_data[f'{emotion}_mean'].mean()
            general_mean = general_data[f'{emotion}_mean'].mean()
            hcp_std = hcp_data[f'{emotion}_std'].mean()
            general_std = general_data[f'{emotion}_std'].mean()
            hcp_n = hcp_data['unique_users'].sum()
            general_n = general_data['unique_users'].sum()

            # Calculate Cohen's d and its CI
            cohen_d_value, ci = calculate_cohen_d_and_ci(
                hcp_mean, 
                general_mean,
                hcp_std,
                general_std,
                hcp_n,
                general_n
            )
            cohen_d_comparison[(phase, emotion)] = (cohen_d_value, ci)

# Update format_output function to include Cohen's d and its CI
def format_output_comparison_with_ci(group_name_1, group_name_2, cohen_d_comparison):
    for (phase, emotion), (cohen_d_value, ci) in cohen_d_comparison.items():
        ci_lower, ci_upper = ci
        print(f"In {phase}, for {emotion.capitalize()}:")
        print(f"- Cohen's d ({group_name_1} vs {group_name_2}): {cohen_d_value:.3f}, 95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")
        print()

# Output the formatted statistics comparing HCP and General populations
format_output_comparison_with_ci('HCP Population', 'General Population', cohen_d_comparison)


In Before COVID Phases, for Sadness:
- Cohen's d (HCP Population vs General Population): 0.009, 95% CI: [-0.065, 0.083]

In Before COVID Phases, for Anger:
- Cohen's d (HCP Population vs General Population): -0.182, 95% CI: [-0.256, -0.108]

In Before COVID Phases, for Surprise:
- Cohen's d (HCP Population vs General Population): -0.329, 95% CI: [-0.403, -0.254]

In Before COVID Phases, for Disgust:
- Cohen's d (HCP Population vs General Population): -0.180, 95% CI: [-0.254, -0.106]

In Before COVID Phases, for Fear:
- Cohen's d (HCP Population vs General Population): 0.166, 95% CI: [0.092, 0.240]

In Before COVID Phases, for Joy:
- Cohen's d (HCP Population vs General Population): 0.101, 95% CI: [0.028, 0.175]

In Phase 1, for Sadness:
- Cohen's d (HCP Population vs General Population): 0.088, 95% CI: [0.014, 0.161]

In Phase 1, for Anger:
- Cohen's d (HCP Population vs General Population): -0.192, 95% CI: [-0.266, -0.118]

In Phase 1, for Surprise:
- Cohen's d (HCP Population vs Gene

In [27]:
import pandas as pd

# Create lists to hold the data for the Excel file
phase_list = []
emotion_list = []
group_list = []
cohen_d_values = []
ci_lower_values = []
ci_upper_values = []

# Iterate over HCP stats and collect data
for index, row in hcp_stats.iterrows():
    phase = row['phase']
    for emotion in emotions:
        cohen_d_value = row[f'{emotion}_cohen_d']
        ci_lower, ci_upper = row[f'{emotion}_cohen_d_ci']
        phase_list.append(phase)
        emotion_list.append(emotion)
        group_list.append('HCP Population')
        cohen_d_values.append(cohen_d_value)
        ci_lower_values.append(ci_lower)
        ci_upper_values.append(ci_upper)

# Iterate over General stats and collect data
for index, row in general_stats.iterrows():
    phase = row['phase']
    for emotion in emotions:
        cohen_d_value = row[f'{emotion}_cohen_d']
        ci_lower, ci_upper = row[f'{emotion}_cohen_d_ci']
        phase_list.append(phase)
        emotion_list.append(emotion)
        group_list.append('General Population')
        cohen_d_values.append(cohen_d_value)
        ci_lower_values.append(ci_lower)
        ci_upper_values.append(ci_upper)

# Iterate over comparison data
for (phase, emotion), (cohen_d_value, ci) in cohen_d_comparison.items():
    ci_lower, ci_upper = ci
    phase_list.append(phase)
    emotion_list.append(emotion)
    group_list.append('HCP vs General Population')
    cohen_d_values.append(cohen_d_value)
    ci_lower_values.append(ci_lower)
    ci_upper_values.append(ci_upper)

# Create a DataFrame
data = {
    "Phase": phase_list,
    "Emotion": emotion_list,
    "Group": group_list,
    "Cohen's d": cohen_d_values,
    "CI Lower": ci_lower_values,
    "CI Upper": ci_upper_values,
}
df = pd.DataFrame(data)

# Save to Excel
output_file_path = "cohen_d_full_results.xlsx"
df.to_excel(output_file_path, index=False)

output_file_path


'cohen_d_full_results.xlsx'

In [16]:
# # Import necessary libraries
# import numpy as np

# # Function to calculate Cohen's d
# def calculate_cohen_d(mean1, mean2, std1, std2, n1, n2):
#     pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
#     d = (mean1 - mean2) / pooled_std
#     return d

# # Compute Cohen's d between HCP and General populations for each phase and emotion
# phases = hcp_stats['phase'].unique()
# emotions = ['sadness', 'anger', 'surprise', 'disgust', 'fear', 'joy']

# cohen_d_comparison = {}

# for phase in phases:
#     for emotion in emotions:
#         hcp_data = hcp_stats[hcp_stats['phase'] == phase]
#         general_data = general_stats[general_stats['phase'] == phase]
        
#         if not hcp_data.empty and not general_data.empty:
#             # Calculate mean and std for HCP and General data
#             hcp_mean = abs(hcp_data[f'{emotion}_mean'].mean() - hcp_stats[hcp_stats['phase'] == "Before COVID Phases"][f'{emotion}_mean'].iloc[0])
#             general_mean = abs(general_data[f'{emotion}_mean'].mean() - general_stats[general_stats['phase'] == "Before COVID Phases"][f'{emotion}_mean'].iloc[0])
#             hcp_std = hcp_data[f'{emotion}_std'].mean()
#             general_std = general_data[f'{emotion}_std'].mean()
#             hcp_n = hcp_data['unique_users'].sum()
#             general_n = general_data['unique_users'].sum()

#             # Calculate Cohen's d between HCP and General populations for the current emotion and phase
#             cohen_d_value = calculate_cohen_d(
#                 hcp_mean, 
#                 general_mean,
#                 hcp_std,
#                 general_std,
#                 hcp_n,
#                 general_n
#             )
#             cohen_d_comparison[(phase, emotion)] = cohen_d_value

# # Update format_output function to include Cohen's d comparisons between populations
# def format_output_comparison(stats_df, group_name, cohen_d_comparison):
#     for index, row in stats_df.iterrows():
#         phase = row['phase']
#         print(f"The top 5 themes for {group_name} in {phase} were:")
#         for emotion in emotions:
#             cohen_d_value = cohen_d_comparison.get((phase, emotion), np.nan)
#             print(f"- {emotion.capitalize()}: Cohen's d (HCP vs General): {cohen_d_value:.3f}")
#         print()

# # Output the formatted statistics with Cohen's d comparisons
# format_output_comparison(hcp_stats, 'HCP Population', cohen_d_comparison)


The top 5 themes for HCP Population in Before COVID Phases were:
- Sadness: Cohen's d (HCP vs General): 0.000
- Anger: Cohen's d (HCP vs General): 0.000
- Surprise: Cohen's d (HCP vs General): 0.000
- Disgust: Cohen's d (HCP vs General): 0.000
- Fear: Cohen's d (HCP vs General): 0.000
- Joy: Cohen's d (HCP vs General): 0.000

The top 5 themes for HCP Population in Phase 1 were:
- Sadness: Cohen's d (HCP vs General): 0.079
- Anger: Cohen's d (HCP vs General): -0.032
- Surprise: Cohen's d (HCP vs General): -0.017
- Disgust: Cohen's d (HCP vs General): -0.002
- Fear: Cohen's d (HCP vs General): 0.391
- Joy: Cohen's d (HCP vs General): 0.034

The top 5 themes for HCP Population in Phase 2 were:
- Sadness: Cohen's d (HCP vs General): 0.124
- Anger: Cohen's d (HCP vs General): 0.027
- Surprise: Cohen's d (HCP vs General): 0.013
- Disgust: Cohen's d (HCP vs General): -0.020
- Fear: Cohen's d (HCP vs General): 0.228
- Joy: Cohen's d (HCP vs General): -0.031

The top 5 themes for HCP Population

In [20]:
# # Import necessary libraries
# import numpy as np
# from scipy.stats import t

# # Function to calculate Cohen's d and its confidence interval
# def calculate_cohen_d_and_ci(mean1, mean2, std1, std2, n1, n2, alpha=0.05):
#     pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
#     d = (mean1 - mean2) / pooled_std

#     # Calculate standard error
#     se_d = np.sqrt((n1 + n2) / (n1 * n2) + d**2 / (2 * (n1 + n2)))

#     # Critical value from t-distribution
#     t_crit = t.ppf(1 - alpha / 2, df=n1 + n2 - 2)

#     # Calculate confidence interval
#     ci_lower = d - t_crit * se_d
#     ci_upper = d + t_crit * se_d

#     return d, (ci_lower, ci_upper)

# # Compute Cohen's d and CI between HCP and General populations for each phase and emotion
# phases = hcp_stats['phase'].unique()
# emotions = ['sadness', 'anger', 'surprise', 'disgust', 'fear', 'joy']

# cohen_d_comparison = {}

# for phase in phases:
#     for emotion in emotions:
#         hcp_data = hcp_stats[hcp_stats['phase'] == phase]
#         general_data = general_stats[general_stats['phase'] == phase]
        
#         if not hcp_data.empty and not general_data.empty:
#             # Calculate mean and std for HCP and General data
#             hcp_mean = abs(hcp_data[f'{emotion}_mean'].mean() - hcp_stats[hcp_stats['phase'] == "Before COVID Phases"][f'{emotion}_mean'].iloc[0])
#             general_mean = abs(general_data[f'{emotion}_mean'].mean() - general_stats[general_stats['phase'] == "Before COVID Phases"][f'{emotion}_mean'].iloc[0])
#             hcp_std = hcp_data[f'{emotion}_std'].mean()
#             general_std = general_data[f'{emotion}_std'].mean()
#             hcp_n = hcp_data['unique_users'].sum()
#             general_n = general_data['unique_users'].sum()

#             # Calculate Cohen's d and its CI
#             cohen_d_value, ci = calculate_cohen_d_and_ci(
#                 hcp_mean, 
#                 general_mean,
#                 hcp_std,
#                 general_std,
#                 hcp_n,
#                 general_n
#             )
#             cohen_d_comparison[(phase, emotion)] = (cohen_d_value, ci)

# # Update format_output function to include Cohen's d and its CI
# def format_output_comparison_with_ci(stats_df, group_name, cohen_d_comparison):
#     for index, row in stats_df.iterrows():
#         phase = row['phase']
#         print(f"The top 5 themes for {group_name} in {phase} were:")
#         for emotion in emotions:
#             cohen_d_value, ci = cohen_d_comparison.get((phase, emotion), (np.nan, (np.nan, np.nan)))
#             ci_lower, ci_upper = ci
#             print(f"- {emotion.capitalize()}: Cohen's d (HCP vs General): {cohen_d_value:.3f}, 95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")
#         print()

# # Output the formatted statistics with Cohen's d and CI comparisons
# format_output_comparison_with_ci(hcp_stats, 'HCP Population', cohen_d_comparison)


The top 5 themes for HCP Population in Before COVID Phases were:
- Sadness: Cohen's d (HCP vs General): 0.000, 95% CI: [-0.074, 0.074]
- Anger: Cohen's d (HCP vs General): 0.000, 95% CI: [-0.074, 0.074]
- Surprise: Cohen's d (HCP vs General): 0.000, 95% CI: [-0.074, 0.074]
- Disgust: Cohen's d (HCP vs General): 0.000, 95% CI: [-0.074, 0.074]
- Fear: Cohen's d (HCP vs General): 0.000, 95% CI: [-0.074, 0.074]
- Joy: Cohen's d (HCP vs General): 0.000, 95% CI: [-0.074, 0.074]

The top 5 themes for HCP Population in Phase 1 were:
- Sadness: Cohen's d (HCP vs General): 0.079, 95% CI: [0.005, 0.153]
- Anger: Cohen's d (HCP vs General): -0.032, 95% CI: [-0.105, 0.042]
- Surprise: Cohen's d (HCP vs General): -0.017, 95% CI: [-0.091, 0.057]
- Disgust: Cohen's d (HCP vs General): -0.002, 95% CI: [-0.076, 0.072]
- Fear: Cohen's d (HCP vs General): 0.391, 95% CI: [0.316, 0.465]
- Joy: Cohen's d (HCP vs General): 0.034, 95% CI: [-0.040, 0.107]

The top 5 themes for HCP Population in Phase 2 were:
- 

In [17]:
# # Import necessary libraries
# import numpy as np

# # Function to calculate Cohen's d
# def calculate_cohen_d(mean1, mean2, std1, std2, n1, n2):
#     pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
#     d = (mean1 - mean2) / pooled_std
#     return d

# # Ensure phases are ordered correctly
# hcp_stats = hcp_stats.sort_values('phase')
# general_stats = general_stats.sort_values('phase')

# # Compute Cohen's d between HCP and General populations for each phase and emotion, compared to the previous phase
# emotions = ['sadness', 'anger', 'surprise', 'disgust', 'fear', 'joy']
# cohen_d_comparison = {}

# for i in range(1, len(hcp_stats)):  # Start from the second phase to compare with the previous one
#     current_phase = hcp_stats['phase'].iloc[i]
#     previous_phase = hcp_stats['phase'].iloc[i - 1]
    
#     for emotion in emotions:
#         # Get HCP and General data for the current and previous phase
#         hcp_data_current = hcp_stats[hcp_stats['phase'] == current_phase]
#         hcp_data_previous = hcp_stats[hcp_stats['phase'] == previous_phase]
#         general_data_current = general_stats[general_stats['phase'] == current_phase]
#         general_data_previous = general_stats[general_stats['phase'] == previous_phase]
        
#         if not hcp_data_current.empty and not general_data_current.empty and not hcp_data_previous.empty and not general_data_previous.empty:
#             # Calculate mean and std for HCP and General data, comparing to the previous phase
#             hcp_mean_current = abs(hcp_data_current[f'{emotion}_mean'].mean() - hcp_data_previous[f'{emotion}_mean'].mean())
#             general_mean_current = abs(general_data_current[f'{emotion}_mean'].mean() - general_data_previous[f'{emotion}_mean'].mean())
#             hcp_std_current = hcp_data_current[f'{emotion}_std'].mean()
#             general_std_current = general_data_current[f'{emotion}_std'].mean()
#             hcp_n_current = hcp_data_current['unique_users'].sum()
#             general_n_current = general_data_current['unique_users'].sum()

#             # Calculate Cohen's d between HCP and General populations for the current emotion and phase
#             cohen_d_value = calculate_cohen_d(
#                 hcp_mean_current, 
#                 general_mean_current,
#                 hcp_std_current,
#                 general_std_current,
#                 hcp_n_current,
#                 general_n_current
#             )
#             cohen_d_comparison[(current_phase, emotion)] = cohen_d_value

# # Update format_output function to include Cohen's d comparisons between populations
# def format_output_comparison(stats_df, group_name, cohen_d_comparison):
#     for index, row in stats_df.iterrows():
#         phase = row['phase']
#         print(f"The top 5 themes for {group_name} in {phase} were:")
#         for emotion in emotions:
#             cohen_d_value = cohen_d_comparison.get((phase, emotion), np.nan)
#             print(f"- {emotion.capitalize()}: Cohen's d (HCP vs General): {cohen_d_value:.3f}")
#         print()

# # Output the formatted statistics with Cohen's d comparisons
# format_output_comparison(hcp_stats, 'HCP Population', cohen_d_comparison)


The top 5 themes for HCP Population in Before COVID Phases were:
- Sadness: Cohen's d (HCP vs General): nan
- Anger: Cohen's d (HCP vs General): nan
- Surprise: Cohen's d (HCP vs General): nan
- Disgust: Cohen's d (HCP vs General): nan
- Fear: Cohen's d (HCP vs General): nan
- Joy: Cohen's d (HCP vs General): nan

The top 5 themes for HCP Population in Phase 1 were:
- Sadness: Cohen's d (HCP vs General): 0.079
- Anger: Cohen's d (HCP vs General): -0.032
- Surprise: Cohen's d (HCP vs General): -0.017
- Disgust: Cohen's d (HCP vs General): -0.002
- Fear: Cohen's d (HCP vs General): 0.391
- Joy: Cohen's d (HCP vs General): 0.034

The top 5 themes for HCP Population in Phase 2 were:
- Sadness: Cohen's d (HCP vs General): -0.052
- Anger: Cohen's d (HCP vs General): -0.041
- Surprise: Cohen's d (HCP vs General): -0.028
- Disgust: Cohen's d (HCP vs General): 0.018
- Fear: Cohen's d (HCP vs General): 0.229
- Joy: Cohen's d (HCP vs General): 0.061

The top 5 themes for HCP Population in Phase 3

In [24]:
# Compute Cohen's d and CI between HCP and General populations for each phase and emotion
emotions = ['sadness', 'anger', 'surprise', 'disgust', 'fear', 'joy']
cohen_d_comparison = {}

# Iterate over each phase and emotion to compare HCP and General populations
for phase in hcp_stats['phase'].unique():
    for emotion in emotions:
        # Get data for the current phase for both populations
        hcp_data = hcp_stats[hcp_stats['phase'] == phase]
        general_data = general_stats[general_stats['phase'] == phase]
        
        if not hcp_data.empty and not general_data.empty:
            # Calculate mean and std for HCP and General data
            hcp_mean = hcp_data[f'{emotion}_mean'].mean()
            general_mean = general_data[f'{emotion}_mean'].mean()
            hcp_std = hcp_data[f'{emotion}_std'].mean()
            general_std = general_data[f'{emotion}_std'].mean()
            hcp_n = hcp_data['unique_users'].sum()
            general_n = general_data['unique_users'].sum()

            # Calculate Cohen's d and its CI
            cohen_d_value, ci = calculate_cohen_d_and_ci(
                hcp_mean, 
                general_mean,
                hcp_std,
                general_std,
                hcp_n,
                general_n
            )
            cohen_d_comparison[(phase, emotion)] = (cohen_d_value, ci)

# Update format_output function to include Cohen's d and its CI
def format_output_comparison_with_ci(group_name_1, group_name_2, cohen_d_comparison):
    for (phase, emotion), (cohen_d_value, ci) in cohen_d_comparison.items():
        ci_lower, ci_upper = ci
        print(f"In {phase}, for {emotion.capitalize()}:")
        print(f"- Cohen's d ({group_name_1} vs {group_name_2}): {cohen_d_value:.3f}, 95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")
        print()

# Output the formatted statistics comparing HCP and General populations
format_output_comparison_with_ci('HCP Population', 'General Population', cohen_d_comparison)


In Before COVID Phases, for Sadness:
- Cohen's d (HCP Population vs General Population): 0.009, 95% CI: [-0.065, 0.083]

In Before COVID Phases, for Anger:
- Cohen's d (HCP Population vs General Population): -0.182, 95% CI: [-0.256, -0.108]

In Before COVID Phases, for Surprise:
- Cohen's d (HCP Population vs General Population): -0.329, 95% CI: [-0.403, -0.254]

In Before COVID Phases, for Disgust:
- Cohen's d (HCP Population vs General Population): -0.180, 95% CI: [-0.254, -0.106]

In Before COVID Phases, for Fear:
- Cohen's d (HCP Population vs General Population): 0.166, 95% CI: [0.092, 0.240]

In Before COVID Phases, for Joy:
- Cohen's d (HCP Population vs General Population): 0.101, 95% CI: [0.028, 0.175]

In Phase 1, for Sadness:
- Cohen's d (HCP Population vs General Population): 0.088, 95% CI: [0.014, 0.161]

In Phase 1, for Anger:
- Cohen's d (HCP Population vs General Population): -0.192, 95% CI: [-0.266, -0.118]

In Phase 1, for Surprise:
- Cohen's d (HCP Population vs Gene